<a href="https://colab.research.google.com/github/slsabilaAura/Tugas-Akhir/blob/main/Preprocessing_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install indoNLP

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 2.8 MB/s eta 0:00:00


In [ ]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.6 MB/s eta 0:00:00


In [ ]:
!pip install langdetect

In [ ]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from collections import defaultdict
from nltk.corpus import stopwords
from collections import Counter
import nltk
from indoNLP.preprocessing import remove_stopwords

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
file_path_expert = "/content/drive/MyDrive/Skripsi2/Dataset/expert.csv"
expert_df = pd.read_csv(file_path_expert)
file_path_proposal = "/content/drive/MyDrive/Skripsi2/Dataset/proposal.csv"
proposal_df = pd.read_csv(file_path_proposal)

In [ ]:
import pandas as pd
import re
import spacy
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import PorterStemmer
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

# Inisialisasi spaCy dan stemmer
nlp = spacy.blank("id")
factory = StemmerFactory()
stemmer_id = factory.create_stemmer()
stemmer_en = PorterStemmer()

# Stopword gabungan: akademik, huruf, angka, bahasa Inggris
custom_stopwords = set([
    # Akademik
    "penelitian", "data", "model", "hasil", "sistem", "metode", "menggunakan",
    "dapat", "dalam", "dengan", "tujuan", "analisis", "penggunaan", "studi",
    "proses", "berdasarkan", "et", "al", "digunakan", "nilai", "informasi",
    "menunjukkan", "hingga", "oleh", "terhadap", "memiliki", "algoritma", "akurasi", "salah",
    "adalah", "di", "ke", "dari", "yaitu", "atau", "lalu", "kemudian", "dkk", "akan", "serta", "ini", "itu",
    "seperti", "telah", "sama", "dan", "yang","dapat","tentu","mana","banyak","ada","buat","oleh","tiap","setiap",
    "lain","selain","amtara","suatu","contoh","beberapa","sehingga","lalu","juga","pilih","pilihan","karena","tahu","diketahui",

    # Huruf a-z dan angka 0-9
    *list("abcdefghijklmnopqrstuvwxyz"),
    *list("0123456789"),

    # Bahasa Inggris umum
    "the", "is", "a", "an", "and", "or", "of", "to", "in", "for", "on", "with",
    "this", "that", "by", "at", "from", "it", "as", "are", "be", "was", "were",
    "has", "have", "had", "not", "but", "we", "they", "their", "our", "its"
])

# Fungsi untuk deteksi bahasa per kata
def detect_language_word(word):
    try:
        return detect(word)
    except LangDetectException:
        return "unknown"

# Fungsi cleaning text
def clean_text(text):
    if pd.isna(text):
        return ''
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"\((?:[A-Z][a-z]+(?:\s+(?:dan|and|&)\s+[A-Z][a-z]+)?)(?:,\s*\d{4}[a-z]?)?\)", "", text)
    text = re.sub(r"\b[A-Z][a-z]+(?:\s+(?:and|dan|&)\s+[A-Z][a-z]+)?\s*,?\s*\d{4}[a-z]?\b", "", text)
    text = re.sub(r"\b[A-Z][a-z]+(?:\s+et\.?\s*al\.?|dkk)\s*,?\s*\d{4}[a-z]?", "", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    text = text.lower()
    text = re.sub(r"\s+", " ", text).strip()
    text = re.sub(r"\b(\w+)-\1\b", r"\1 \1", text)
    text = re.sub(r"\b(dasar|landasan)\s+teori\b", "", text, flags=re.IGNORECASE)
    return text

# Preprocessing langsung: cleaning → tokenizing → stopword removal → stemming (ID & EN)
def preprocess_text_detail(text):
    original = text
    tokens = [token.text.lower() for token in nlp(original)]
    tokens_no_stopwords = [t for t in tokens if t not in custom_stopwords and t.strip() != ""]

    stemmed = []
    for token in tokens_no_stopwords:
        lang = detect_language_word(token)
        if lang == 'en':
            stemmed_token = stemmer_en.stem(token)
        elif lang == 'id':
            stemmed_token = stemmer_id.stem(token)
        else:
            stemmed_token = stemmer_id.stem(token)  # default ID
        stemmed.append(stemmed_token)

    return {
        "original_text": original,
        "tokens": tokens,
        "no_stopwords": tokens_no_stopwords,
        "stemmed": stemmed,
        "final_text": " ".join(stemmed)
    }

# Pipeline preprocessing lengkap
def preprocess_pipeline(file_path_proposal, file_path_expert):
    proposal_df = pd.read_csv(file_path_proposal)
    expert_df = pd.read_csv(file_path_expert)

    proposal_df["full_text"] = proposal_df["proposal_introduction"].astype(str) + " " + proposal_df["proposal_theory"].astype(str)
    expert_df["full_text"] = expert_df["research_abstract"].astype(str)

    print("🚀 Preprocessing proposal...")
    proposal_results = proposal_df["full_text"].apply(preprocess_text_detail)
    proposal_df["original_text"] = proposal_results.apply(lambda x: x["original_text"])
    proposal_df["tokens"] = proposal_results.apply(lambda x: x["tokens"])
    proposal_df["no_stopwords"] = proposal_results.apply(lambda x: x["no_stopwords"])
    proposal_df["stemmed"] = proposal_results.apply(lambda x: x["stemmed"])
    proposal_df["processed_text"] = proposal_results.apply(lambda x: x["final_text"])

    print("🚀 Preprocessing publikasi dosen...")
    expert_results = expert_df["full_text"].apply(preprocess_text_detail)
    expert_df["original_text"] = expert_results.apply(lambda x: x["original_text"])
    expert_df["tokens"] = expert_results.apply(lambda x: x["tokens"])
    expert_df["no_stopwords"] = expert_results.apply(lambda x: x["no_stopwords"])
    expert_df["stemmed"] = expert_results.apply(lambda x: x["stemmed"])
    expert_df["processed_text"] = expert_results.apply(lambda x: x["final_text"])

    print("✅ Preprocessing selesai!")
    return proposal_df, expert_df


In [ ]:
proposal_df, expert_df = preprocess_pipeline(file_path_proposal, file_path_expert)


🚀 Preprocessing proposal...
🚀 Preprocessing publikasi dosen...
✅ Preprocessing selesai!


In [ ]:
print(proposal_df)

    proposal_id student_id                                     proposal_title  \
0            P1         S1  Analisa dan Implementasi Steganografi dengan M...   
1           P10        S10  Clustering Penyalahguna Narkotika Menggunakan ...   
2           P11        S11  CNN Berbasis Blok Xception Menggunakan Fitur K...   
3           P12        S12  COLOR-EMBEDDED-GRAYSCALE IMAGE MENGGUNAKAN RES...   
4           P13        S13  COMPREHENSIVE MULTIPLE SECRET SHARING MENGGUNA...   
..          ...        ...                                                ...   
137        P138       S138  Pengaruh Double Learning Logistic Regression p...   
138        P139       S139  Residual Model pada Detail Retaining Convoluti...   
139        P140       S140  Analisa Sentimen Komentar terhadap Kebijakan P...   
140        P141       S141  Klasifikasi Relevansi Trending Topic Dengan Ko...   
141        P142       S142  Analisa Perbandingan Pemodelan TF-IDF dan WORD...   

                           

In [ ]:
# prompt: how to export to csv

# Save the preprocessed DataFrames to new CSV files
proposal_df.to_csv('/content/drive/MyDrive/Skripsi2/Dataset/processed_proposalC.csv', index=False)
expert_df.to_csv('/content/drive/MyDrive/Skripsi2/Dataset/processed_expertBC.csv', index=False)

print("Processed dataframes saved to CSV in Google Drive.")

Processed dataframes saved to CSV in Google Drive.
